The following code was taken by the repository of Harry Stuart. https://github.com/harrystuart/tfworldhackathon.

This is an implementation of WaveGAN architecture to generate audio files of 4 seconds. In order to  generate audio for 10 genres, this scrit is supposed to be run 10 times, replacing the value of variable "INSTRUMENT" for respective  genre.

In [ ]:
# Import modules
import tensorflow as tf
import numpy as np
import librosa
import random
import os
import soundfile as sf
import model

In [ ]:
#Code to load google drive
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instantiate model
gan = model.GAN(model_dims=MODEL_DIMS, num_samples=NUM_SAMPLES, 
              gradient_penalty_weight=GRADIENT_PENALTY_WEIGHT, instrument=INSTRUMENT,
              noise_len=NOISE_LEN, batch_size=BATCH_SIZE, sr=Fs)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense (Dense)               (None, 16384)             1654784   
                                                                 
 reshape (Reshape)           (None, 16, 1024)          0         
                                                                 
 re_lu (ReLU)                (None, 16, 1024)          0         
                                                                 
 lambda (Lambda)             (None, 1, 16, 1024)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 1, 64, 512)       13107712  
 nspose)                                                         
                                                             

In [ ]:
# Create training data

X_train = []
for file in os.listdir(DATA_DIR): ### Modify for your data directory
    with open(DATA_DIR + fr"/{file}", "rb") as f:
        samples, _ = librosa.load(f, sr = Fs)
        # Pad short audio files to NUM_SAMPLES duration
        if len(samples) < NUM_SAMPLES:
            audio = np.array([np.array([sample]) for sample in samples])
            padding = np.zeros(shape=(NUM_SAMPLES - len(samples), 1), dtype='float32')
            X_train.append(np.append(audio, padding, axis=0))
        # Create slices of length NUM_SAMPLES from long audio
        else:
            p = len(samples) // (NUM_SAMPLES)
            for i in range(p - 1):
                sample = np.expand_dims(samples[i*NUM_SAMPLES:(i+1)*NUM_SAMPLES], axis=1)
                X_train.append(sample)

print(f"X_train shape = {(len(X_train),) + X_train[0].shape}")

X_train shape = (600, 65536, 1)


In [ ]:
# Save some random training data slices and create baseline generated data for comparison
for i in range(50):  
    sf.write(f"/content/drive/MyDrive/Outputs/{INSTRUMENT}/real-{i}.wav", X_train[random.randint(0, len(X_train) - 1)],samplerate=Fs)


In [ ]:
# Save some random data slices as fake for comparison
gan.sample("fake")
train_summary_writer = tf.summary.create_file_writer("logs/train")

In [ ]:
# Train GAN
with train_summary_writer.as_default():
    steps_per_epoch = len(X_train) // BATCH_SIZE 

    for e in range(EPOCHS):
        for i in range(steps_per_epoch):
            D_loss_sum = 0
        
            # Update dcritic a set number of times for each update of the generator
            for n in range(D_UPDATES_PER_G_UPDATE):
                gan.D.reset_states()
                D_loss_dict = gan.train_D(np.array(random.sample(X_train, BATCH_SIZE)))
                D_loss_sum += D_loss_dict['d_loss']
            
            # Calculate average loss of critic for current step
            D_loss = D_loss_sum / D_UPDATES_PER_G_UPDATE
            
            G_loss_dict = gan.train_G()
            G_loss = G_loss_dict['g_loss']
        
            # Write logs
            tf.summary.scalar('d_loss', D_loss, step=(e*steps_per_epoch)+i)
            tf.summary.scalar('g_loss', G_loss, step=(e*steps_per_epoch)+i)
        
            print(f"step {(e*steps_per_epoch)+i}: d_loss = {D_loss} g_loss = {G_loss}")
        
        # Periodically sample generator
        if e % EPOCHS_PER_SAMPLE == 0:
            gan.sample(e)